In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

In [12]:
# set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [16]:
# Function to load and prepare your data

def load_data():

  data = pd.read_csv(f'feature_engineered_dataset.csv')

  X = data.drop(columns = ['remainder__Decided to Pursue MBA?'])
  y = data['remainder__Decided to Pursue MBA?']

  print("Please replace the load_data function with your actual data loading code")
  return None, None

In [20]:
# Function to create a model with given hyperparameters

def create_model(params):

    model = Sequential()

    # Input layer
    model.add(Dense(
        units = params['units_1'],
        activation = params['activation'],
        kernel_regularizer = tf.keras.regularizers.l1_l2(l1 = params['l1'], l2 = params['l2']),
        input_dim = X_train.shape[1]
    ))

    # Add dropout after first layer
    model.add(Dropout(params['dropout_1']))

    # Hidden layer
    for i in range(params['n_layers']):
        model.add(Dense(
            units = params[f'units_{i+2}'],
            activation = params['activation'],
            kernel_regularizer = tf.keras.regularizers.l1_l2(l1 = params['l1'], l2 = params['l2'])
        ))
        model.add(Dropout(params[f'dropout_{i+1}']))
    # Output layer (binary classification)
    model.add(Dense(1, activation = 'sigmoid'))

    #compile the model
    optimizer_name = params['optimizer']
    learning_rate = params['learning_rate']

    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate = learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate = learning_rate, momentum = params['momentum'])
    else:
        optimizer = RMSprop(learning_rate = learning_rate)

    model.compile(
        optimizer = optimizer,
        loss = 'binary_crossentropy',
        metrics = ['accuracy', tf.keras.metrics.AUC(name = 'auc')]
    )
    return model
        

In [22]:
# Object function for Optuna

def objective(trial):
    # Define hyperparametrs to tune
    params = {
        #network architecture
        'n_layers': trial.suggest_int('n_layers', 1, 3),
        'units_1': trial.suggest_int('units_1', 16, 256),
        'units_2': trial.suggest_int('units_2', 16, 256),
        'units_3': trial.suggest_int('units_3', 16, 256),
        'units_4': trial.suggest_int('units_4', 16, 256),
        'dropout_1': trial.suggest_float('dropout_1', 0.0, 0.5),
        'dropout_2': trial.suggest_float('dropout_2', 0.0, 0.5),
        'dropout_3': trial.suggest_float('dropout_3', 0.0, 0.5),
        'dropout_4': trial.suggest_float('dropout_4', 0.0, 0.5),
        'activation': trial.suggest_categorical('activation', ['relu', 'elu', 'selu']),

        # Regularization
        'l1': trial.suggest_float('l1', 1e-8, 1e-3, log = True),
        'l2': trial.suggest_float('l2', 1e-8, 1e-3, log = True),

        #Optimization
        'optimizer': trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop']),
        'learnig_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log = True),
        'mometum': trial.suggest_float('momentum', 0.0, 0.99),
        'batch_size': trial.suggest_categorical('batch_size', [16, 32, 64, 128]),
        'epochs': 100, # Fixed number of maximum epochs with early stopping
        
    }


    # use stratified k-fold cross-validation
    n_folds = 5
    skf = StratifiedKFold(n_splits = n_folds, shuffle = True, random_state = 42)
    cv_scores = []

    for train_idx, val_idx in skf.split(X_train_scaled, y_train):
        X_fold_train, X_fold_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
        y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]

        # calculate class weights if imbalance
        class_weights = compute_class_weight(
            class_weight = 'balanced',
            classes = np.unique(y_fold_train),
            y = y_fold_train
        )
        class_weight_dict = {i: weight for i, weight in enumerated(class_weights)}

        # Create and train model
        model = create_model(params)

        # Define callbacks
        early_stopping = EarlyStopping(
            monitor = 'val_auc',
            patience = 10,
            restore_best_weights = True,
            mode = 'max'
        )

        reduce_lr = ReduceLROnPlateau(
            monitor = 'val_loss',
            factor = 0.2,
            patience = 5, 
            min_lr = 1e-6
        )

        # Train the model
        history = model.fit(
            X_fold_train, y_fold_train,
            epochs = params['epochs'],
            batch_size = params['batch_size'],
            validation_data = (X_fold_val, y_fold_val),
            class_weight = calss_weight_dict,
            callbacks = [early_stopping, reduce_lr],
            verbose = 0
        )

        # Evaluate on validation set
        y_pred_proba = model.predict(X_fold_val, verbose = 0)
        y_preds = (y_pred_proba > 0.5).astype(int)

        # Get AUC score
        auc = roc_auc_score(y_fold_val, y_pred_proba)
        cv_scores.append(auc)

        # Free up score
        tf.keras.backend.clear_session()
        
    # Return the mean AUC score across all fold
    return np.mean(cv_scores)   